# Foley Ch. 5 (Model 1): Delay model of production, sales, and capitals

## Definitions (flows)
\[
\begin{aligned}
P(t) &= C(t - T_P) \\
S(t) &= (1+q)\,P(t - T_R) \\
S'(t) &= P(t - T_R) \\
S''(t) &= q\,P(t - T_R) \\
C(t) &= S'(t - T_F) + p\,S''(t - T_F)
\end{aligned}
\]

## Capital accumulation
\[
\begin{aligned}
\frac{dN}{dt} &= C(t) - P(t) \\
\frac{dX}{dt} &= P(t) - S'(t) \\
\frac{dF}{dt} &= S'(t) + pS''(t) - C(t)
\end{aligned}
\]

## Variables (Foley names)
- \(N(t)\): Productive Capital  
- \(X(t)\): Commercial Capital  
- \(F(t)\): Financial Capital  
- \(C(t)\): Capital Outlays  
- \(P(t)\): Finished Product  
- \(S(t)\): Sales  


## Implicit boundary condition (Foley)

To obtain a smooth exponential path, we assume an exponential history for capital outlays:

\[
C(t)=C(0)e^{gt} \quad \text{for all } t \le 0,\qquad C(0)=1.
\]

Given the delay identities, this history is consistent only if \(g\) satisfies the characteristic condition
\[
C(t)=(1+pq)\,C\!\left(t-(T_P+T_R+T_F)\right),
\]
which implies
\[
1=(1+pq)\,e^{-g(T_P+T_R+T_F)}
\quad\Rightarrow\quad
g=\frac{\ln(1+pq)}{T_P+T_R+T_F}.
\]

We also take \(P(t)=0\) for \(t<0\) as a pre-history, but with exponential \(C(t)\) the model’s defining equation
\(P(t)=C(t-T_P)\) generates an exponential \(P(t)\) for \(t\ge 0\).


## Optional extension: full-employment constraint on variable capital (not in Foley)

In Foley’s analysis, capital outlays \(C(t)\) are unconstrained and may grow exponentially along the balanced growth path. Foley also decomposes capital outlays into:

\[
\text{Variable capital outlays} = k\,C(t), \qquad
\text{Constant capital outlays} = (1-k)\,C(t),
\]
with \(0 \le k \le 1\).

As an optional extension (not present in Foley’s original analysis), we allow for a *hard full-employment constraint* on variable capital outlays. Specifically, we impose
\[
k\,C(t) \le \bar V,
\]
where \(\bar V\) is an exogenously given ceiling corresponding to full employment.

In the model, this constraint is implemented by replacing the unconstrained outlays \(C_{\mathrm{raw}}(t)\) implied by the delay structure with an effective outlay
\[
C_{\mathrm{eff}}(t) = \min\!\left\{ C_{\mathrm{raw}}(t),\, \frac{\bar V}{k} \right\}.
\]

When the constraint does not bind, the model reproduces Foley’s unconstrained exponential path. Once the constraint binds, capital outlays become capped, introducing a regime change that generally destroys the pure exponential solution and propagates through the production–sales–finance delays.

This extension is intended purely for exploration and is not part of Foley’s original theoretical construction.


In [ ]:
import sys

if sys.platform == "emscripten":
    try:
        import ipywidgets  # noqa: F401
    except Exception:
        import piplite
        await piplite.install(["ipywidgets"])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, Markdown

def _interp(ts, ys, tq, left_value=0.0):
    if tq < ts[0]:
        return left_value
    if tq >= ts[-1]:
        return ys[-1]
    i = np.searchsorted(ts, tq) - 1
    t0, t1 = ts[i], ts[i+1]
    y0, y1 = ys[i], ys[i+1]
    w = (tq - t0) / (t1 - t0)
    return (1 - w) * y0 + w * y1


def simulate_foley_ch5_part1(
    T=120.0,
    dt=0.05,
    T_P=2.0,
    T_R=5.0,
    T_F=3.0,
    q=0.2,
    p=0.5,
    k=0.6,
    full_employment_cap=False,
    Vbar=1.0,
):
    """
    Foley Ch.5 (first part) with exponential prehistory and an optional
    (non-Foley) hard full-employment constraint on variable capital outlays.

    Flows:
      P(t)  = C(t - T_P)
      S(t)  = (1+q) P(t - T_R)
      S'(t) = P(t - T_R)
      S''(t)= q P(t - T_R)
      C(t)  = S'(t - T_F) + p S''(t - T_F)

    Capitals:
      dN/dt = C(t) - P(t)
      dX/dt = P(t) - S'(t)
      dF/dt = S'(t) + p S''(t) - C(t)

    Exponential prehistory (implicit in Foley's exponential-path discussion):
      C(t) = C(0) e^{g t} for all t <= 0, with C(0)=1 and
      g = ln(1 + p q) / (T_P + T_R + T_F).

    Optional extension (not in Foley):
      Enforce k C(t) <= Vbar by capping C(t) at Vbar/k when enabled.
    """

    # --- time grid ---
    if dt <= 0:
        raise ValueError("dt must be positive.")
    n = int(np.floor(T / dt)) + 1
    t = np.linspace(0.0, dt * (n - 1), n)

    # --- arrays: flows ---
    C = np.zeros(n)      # Capital Outlays
    P = np.zeros(n)      # Finished Product
    Spr = np.zeros(n)    # S'(t)
    Sdp = np.zeros(n)    # S''(t)
    S = np.zeros(n)      # Sales

    # --- arrays: capitals ---
    N = np.zeros(n)      # Productive Capital
    X = np.zeros(n)      # Commercial Capital
    F = np.zeros(n)      # Financial Capital

    # --- exponential prehistory parameters ---
    C0 = 1.0
    C[0] = C0

    T_cycle = T_P + T_R + T_F
    gain = 1.0 + p * q
    if T_cycle <= 0:
        raise ValueError("Need T_P + T_R + T_F > 0 for exponential prehistory.")
    if gain <= 0:
        raise ValueError("Need 1 + p*q > 0 for exponential prehistory (log defined).")
    g = float(np.log(gain) / T_cycle)

    # --- full-employment cap on variable outlays k*C ---
    C_cap = None
    if full_employment_cap:
        if k < 0 or k > 1:
            raise ValueError("k must lie in [0,1].")
        if k > 0:
            if Vbar < 0:
                raise ValueError("Vbar must be nonnegative.")
            C_cap = float(Vbar / k)
        else:
            # k == 0 => no variable capital => constraint irrelevant
            C_cap = None

    # --- history/interpolation accessors ---
    def C_hist(tq):
        # exponential prehistory for tq <= 0
        return C0 * np.exp(g * tq)

    def C_at(tq):
        if tq <= 0:
            return C_hist(tq)
        # For tq > 0, interpolate from computed series
        return _interp(t, C, tq, left_value=C0)

    def P_at(tq):
        # Use definitional relation everywhere: P(t) = C(t - T_P)
        return C_at(tq - T_P)

    # --- simulation loop ---
    for i, ti in enumerate(t):
        # Finished product
        P[i] = C_at(ti - T_P)

        # Sales components
        P_for_sales = P_at(ti - T_R)
        Spr[i] = P_for_sales
        Sdp[i] = q * P_for_sales
        S[i] = (1.0 + q) * P_for_sales

        if i > 0:
            # Raw outlays from the delay loop:
            # C(t) = (1 + p q) C(t - T_cycle)
            C_raw = gain * C_at(ti - T_cycle)

            # Optional hard cap from full employment
            if C_cap is not None:
                C[i] = min(C_raw, C_cap)
            else:
                C[i] = C_raw

            # Integrate capitals (Euler)
            dN = C[i] - P[i]
            dX = P[i] - Spr[i]
            dF = Spr[i] + p * Sdp[i] - C[i]

            N[i] = N[i-1] + dt * dN
            X[i] = X[i-1] + dt * dX
            F[i] = F[i-1] + dt * dF

    # Derived decomposition of outlays
    V = k * C                  # variable capital outlays
    Kconst = (1.0 - k) * C     # constant capital outlays

    return {
        "t": t,
        "C": C, "P": P, "S": S, "S'": Spr, "S''": Sdp,
        "N": N, "X": X, "F": F,
        "V": V, "Kconst": Kconst,
        "g": g, "T_cycle": T_cycle, "gain": gain,
        "C_cap": C_cap,
    }


In [ ]:
text = widgets.HTML()
out = widgets.Output()

T = widgets.FloatSlider(value=120.0, min=20.0, max=400.0, step=10.0, description="T", continuous_update=False)
dt = widgets.FloatSlider(value=0.05, min=0.01, max=0.5, step=0.01, description="dt", continuous_update=False)

T_P = widgets.FloatSlider(value=2.0, min=0.0, max=20.0, step=0.5, description="T_P", continuous_update=False)
T_R = widgets.FloatSlider(value=5.0, min=0.0, max=50.0, step=0.5, description="T_R", continuous_update=False)
T_F = widgets.FloatSlider(value=3.0, min=0.0, max=20.0, step=0.5, description="T_F", continuous_update=False)

q = widgets.FloatSlider(value=0.2, min=0.0, max=5.0, step=0.05, description="q", continuous_update=False)
p = widgets.FloatSlider(value=0.5, min=0.0, max=5.0, step=0.05, description="p", continuous_update=False)

k = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.01, description="k", continuous_update=False)

Vbar = widgets.FloatSlider(
    value=1.0, min=0.0, max=50.0, step=0.1,
    description="V̄", continuous_update=False
)

full_emp = widgets.ToggleButton(
    value=False,
    description="Full employment cap",
    tooltip="Impose k·C(t) ≤ V̄ by capping C(t) at V̄/k",
    button_style=""
)


def redraw(*args):
    # Update derived quantities text
    cycle_delay = float(T_P.value + T_R.value + T_F.value)
    gain = float(1 + p.value * q.value)
    g_val = np.log(gain) / cycle_delay if (cycle_delay > 0 and gain > 0) else float("nan")
    f"Growth rate: g = ln(1+p·q)/T_cycle = {g_val:.5f}<br>"

    cap_info = ""
    if full_emp.value and k.value > 0:
        Ccap = Vbar.value / k.value
        cap_info = f"Full-employment cap: k·C ≤ V̄ ⇒ C ≤ V̄/k = {Ccap:.3f}<br>"
    else:
        cap_info = "Full-employment cap: off<br>"


    # Heuristic resolution indicator for delays
    min_delay = min(float(T_P.value), float(T_R.value), float(T_F.value)) if (T_P.value > 0 and T_R.value > 0 and T_F.value > 0) else None
    if min_delay is None:
        res_note = "Delay resolution note: one or more delays is zero."
    else:
        steps_per_min_delay = min_delay / float(dt.value) if dt.value > 0 else float("inf")
        if steps_per_min_delay < 5:
            res_note = f"Delay resolution warning: only ~{steps_per_min_delay:.1f} steps across the smallest delay."
        else:
            res_note = f"Delay resolution: ~{steps_per_min_delay:.1f} steps across the smallest delay."

    text.value = (
        "<b>Derived quantities</b><br>"
        f"Cycle delay: T_cycle = T_P + T_R + T_F = {cycle_delay:.2f}<br>"
        f"Gain factor: g = 1 + p·q = {gain:.3f}<br>"
        f"{cap_info}"
        f"{res_note}"
    )

    with out:
        out.clear_output(wait=True)
        res = simulate_foley_ch5_part1(
            T=float(T.value),
            dt=float(dt.value),
            T_P=float(T_P.value),
            T_R=float(T_R.value),
            T_F=float(T_F.value),
            q=float(q.value),
            p=float(p.value),
            k=float(k.value),
            full_employment_cap=bool(full_emp.value),
            Vbar=float(Vbar.value),
        )

        t_arr = res["t"]

        plt.figure()
        plt.plot(t_arr, res["C"], label="C(t) Capital Outlays")
        plt.plot(t_arr, res["P"], label="P(t) Finished Product")
        plt.plot(t_arr, res["S'"], label="S'(t)")
        plt.plot(t_arr, res["S''"], label="S''(t)")
        plt.plot(t_arr, res["S"], label="S(t) Sales")
        plt.legend()
        plt.xlabel("t")
        plt.title("Flows")
        plt.show()

        plt.figure()
        plt.plot(t_arr, res["N"], label="N(t) Productive Capital")
        plt.plot(t_arr, res["X"], label="X(t) Commercial Capital")
        plt.plot(t_arr, res["F"], label="F(t) Financial Capital")
        plt.legend()
        plt.xlabel("t")
        plt.title("Capitals")
        plt.show()

        plt.figure()
        plt.plot(t_arr, res["V"], label="k·C(t)  (variable capital outlays)")
        plt.plot(t_arr, res["Kconst"], label="(1-k)·C(t)  (constant capital outlays)")
        if res["C_cap"] is not None:
            plt.plot(t_arr, np.full_like(t_arr, Vbar.value), label="V̄ (full employment ceiling)")
        plt.legend()
        plt.xlabel("t")
        plt.title("Decomposition of C(t)")
        plt.show()


ui = widgets.VBox([
    widgets.HBox([T, dt]),
    widgets.HBox([T_P, T_R, T_F]),
    widgets.HBox([q, p]),
    widgets.HBox([k, Vbar, full_emp]),
    text
])


for w in [T, dt, T_P, T_R, T_F, q, p, k, Vbar, full_emp]:
    w.observe(redraw, names="value")

display(ui, out)
redraw()
